# Generate Encord Active Model Predictions Using Encord Annotate Micro-Models

#### Note: Ontologies of the Encord Active and Encord Annotate should be the same

## Set parameters

In [ ]:
ENCORD_ACTIVE_PROJECT_PATH='/path/to/local/encord_active/project'
ENCORD_SSH_KEY_PATH='ENCORD_SSH_KEY_PATH'
ENCORD_PROJECT_HASH='ENCORD_PROJECT_HASH'
ENCORD_MODEL_ITERATION_HASH='ENCORD_MODEL_ITERATION_HASH'

BATCH_SIZE=24
CONFIDENCE_THRESHOLD=0.6
IOU_THRESHOLD=0.3

## Import libraries

In [ ]:
import os
import json
import pickle
from pathlib import Path

from encord import EncordUserClient, Project
from tqdm import tqdm

from encord_active.lib.common.iterator import DatasetIterator
from encord_active.lib.db.predictions import (
    BoundingBox,
    Format,
    ObjectDetection,
    Prediction,
)
from encord_active.lib.project.project_file_structure import ProjectFileStructure

In [ ]:
user_client = EncordUserClient.create_with_ssh_private_key(Path(ENCORD_SSH_KEY_PATH).expanduser().read_text())
project: Project = user_client.get_project(ENCORD_PROJECT_HASH)

ea_project_fs: ProjectFileStructure = ProjectFileStructure(ENCORD_ACTIVE_PROJECT_PATH)
iterator = DatasetIterator(ea_project_fs.project_dir)

ontology = json.loads(ea_project_fs.ontology.read_text(encoding="utf-8"))
ontology_featureHashes = [obj["featureNodeHash"] for obj in ontology.get("objects")]

predictions_to_store = []
file_paths = []
data_units = []

pbar = tqdm(total=iterator.length, desc="Running inference", leave=True)
for counter, (data_unit, _) in enumerate(iterator.iterate()):

    file_paths.append(
        next(
            ea_project_fs.label_row_structure(iterator.label_hash).iter_data_unit(data_unit["data_hash"])
        ).path.as_posix()
    )
    data_units.append(data_unit)

    if (counter + 1) % BATCH_SIZE == 0 or counter + 1 == iterator.length:
        inference_results = project.model_inference(
            ENCORD_MODEL_ITERATION_HASH,
            file_paths=file_paths,
            conf_thresh=CONFIDENCE_THRESHOLD,
            iou_thresh=IOU_THRESHOLD,
        )

        for inference_result, du in zip(inference_results, data_units):
            for obj in inference_result["predictions"]["0"]["objects"]:
                
                if obj["shape"] != "bounding_box":
                    print(f"prediction is not in `bbox` type (it is in {obj['shape']} type), skipping...")
                    continue
                
                if obj["featureHash"] not in ontology_featureHashes:
                    print(
                        f"'{obj['name']}' with featureHash '{obj['featureHash']}' is not available in the ontology of"
                        f" the Encord Active project."
                    )
                    continue

                prediction = Prediction(
                    data_hash=du["data_hash"],
                    confidence=obj["confidence"],
                    object=ObjectDetection(
                        format=Format.BOUNDING_BOX,
                        data=BoundingBox(
                            x=obj["bounding_box"]["x"],
                            y=obj["bounding_box"]["y"],
                            w=obj["bounding_box"]["w"],
                            h=obj["bounding_box"]["h"],
                        ),
                        feature_hash=obj["featureHash"],
                    ),
                )
                predictions_to_store.append(prediction)

        file_paths = []
        data_units = []

    pbar.update(1)

prediction_file = f"predictions_{ENCORD_MODEL_ITERATION_HASH[:8]}.pkl"
with open((ea_project_fs.project_dir / prediction_file), "wb") as f:
    pickle.dump(predictions_to_store, f)

In [ ]:
os.chdir(ea_project_fs.project_dir.as_posix()) 
! encord-active import predictions {prediction_file}

print("\n✅︎ Predictions are imported successfully!")